In [ ]:
#@title <-- Press Play to trigger setup { run: "auto", display-mode: "form" }
#@markdown - Autenticates user, initialises BigQuery `client`
#@markdown - Sets default `--project` to `PROJECT_ID` and adds  `--no_df` and `--param` flags to `%%bigquery`
#@markdown - Ask for user  confirmation before executing queries that will cost more than `COST_LIMIT_USD`, unless `--cost_approved` flag is specified
from datetime import datetime
from google.cloud import bigquery
import pandas as pd
import json
import matplotlib.pyplot as plt
from cartoframes.viz import Map, Layer, color_continuous_style, basic_style
import numpy as np
import difflib
client = bigquery.Client(project='uc-prox-core-dev')

#!pip install plotly==4.8

  # Quick way to install hind font
!npm install -g npm
!npm install -g google-font-installer
!gfi install hind -v 300

import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
import plotly.offline as py
import plotly.express as px
from plotly.subplots import make_subplots

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import IPython.display
from IPython.display import display, clear_output

#define colors for printing
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

pd.options.plotting.backend = "plotly"

# create unacast template (corporate design)
pio.templates['unacast'] = go.layout.Template({
    'data': {'scatter': [{'marker': {'size': 20, 'symbol': 'hexagon'}, 'type': 'scatter'}]},
    'layout': {'autosize': True,
               'colorscale': {'diverging': [[0, '#8e0152'], [0.1, '#c51b7d'],
                                            [0.2, '#de77ae'], [0.3, '#f1b6da'],
                                            [0.4, '#fde0ef'], [0.5, '#f7f7f7'],
                                            [0.6, '#e6f5d0'], [0.7, '#b8e186'],
                                            [0.8, '#7fbc41'], [0.9, '#4d9221'], [1,
                                            '#276419']],
                              'sequential': [[0.0, '#0d0887'],
                                             [0.1111111111111111, '#46039f'],
                                             [0.2222222222222222, '#7201a8'],
                                             [0.3333333333333333, '#9c179e'],
                                             [0.4444444444444444, '#bd3786'],
                                             [0.5555555555555556, '#d8576b'],
                                             [0.6666666666666666, '#ed7953'],
                                             [0.7777777777777778, '#fb9f3a'],
                                             [0.8888888888888888, '#fdca26'], [1.0,
                                             '#f0f921']],
                              'sequentialminus': [[0.0, '#0d0887'],
                                                  [0.1111111111111111, '#46039f'],
                                                  [0.2222222222222222, '#7201a8'],
                                                  [0.3333333333333333, '#9c179e'],
                                                  [0.4444444444444444, '#bd3786'],
                                                  [0.5555555555555556, '#d8576b'],
                                                  [0.6666666666666666, '#ed7953'],
                                                  [0.7777777777777778, '#fb9f3a'],
                                                  [0.8888888888888888, '#fdca26'],
                                                  [1.0, '#f0f921']]},
               'colorway': ['#FF8000', '#1A4D60', '#00B39B', '#34768F', '#C42240',
                            '#C42240', '#404040'], # removed , '#F6F6F6' for better plotting (white)
               'font': {'color': '#1C1C1C', 'family': 'Hind'},
               'hoverlabel': {'align': 'left'},

               'height': 600,
               'width': 1400,
               'images': [{'sizex': 0.3,
                           'sizey': 0.3,
                           'source': ('https://proptechzone.com/wp-content/uploads/2019/10/unacast-3b04492b-4fe9-4b70-9dc7-43fb22aa6d5c-1.png'),
                           #'source': "https://global-uploads.webflow.com/5dc3e2af6a906d9cc232e1bc/5dc3e2af6a906dceb532e551_unacast-symbol-uc_orange-rgb.png",
                           'x': 0.96,
                           'xanchor': 'center',
                           'xref': 'paper',
                           'y': 0.9,
                           'yanchor': 'bottom',
                           'yref': 'paper',
                           'name': 'uc-logo'}],
               'margin': {'b': 60, 'l': 60, 'r': 60, 't': 60},
               'paper_bgcolor': '#FFFFFF',
               'plot_bgcolor': '#F5F6F7',
               'title': {'font': {'size': 25}},
              'xaxis': {'automargin': True,
                         'gridcolor': 'white',
                         'linecolor': 'white',
                         'showgrid': True,
                         'tickcolor': 'rgb(51,51,51)',
                         'ticks': 'outside',
                         'title': {'standoff': 15},
                         'zerolinecolor': 'white'},
               'yaxis': {'automargin': True,
                         'gridcolor': 'white',
                         'linecolor': 'white',
                         'showgrid': True,
                         'tickcolor': 'rgb(51,51,51)',
                         'ticks': 'outside',
                         'title': {'standoff': 15},
                         'zerolinecolor': 'white'}}
})


# Print success message

print("Ran at", datetime.now(), "GMT")

In [ ]:
import warnings
import plotly.express as px
import re
from cartoframes.viz import Map, Layer, color_continuous_style
warnings.simplefilter("once")

#The aim of this notebook
is to compare SafeGraph and Unacast patterns to show where those versions align and where they differ. This should be used to track process for key industries as well.

Reproducibility given by consistent analysis:
- restrict to target industries: ('722511','722513','444130','445110','445120','447110','452319','517312','713990','721110','721120','722515')
- restrict to 2021-01-01 to 2021-03-31
- region: Chicago + Dallas
- for trimmed mean remove top and low 0.01 quantile
- percentage of places above .55 correlation used

#Todo:

- spider plot: 
  - run grand summary per naics code
  - adjust radar plot to use that output
  - add more metrixs (percent of placekey with sign. drop)
- how many NaN and 0 (what are we dropping)


In [ ]:
#@title Input tables and filters
SG_table = "uc-prox-core-dev.viking_input.combined_weekly_patterns" #@param {type: "string"}
UC_table = "uc-prox-core-dev.viking_align_v1_patterns.patterns_final_view_weekly" #@param {type: "string"}
PLACES_table = "uc-prox-core-dev.viking_input.combined_places" #@param{type: "string"}

#@markdown restrict to specific industries or in time:
industries = "('722511','722513','444130','445110','445120','447110','452319','517312','713990','721110','721120','722515')" #@param {type: "string"}
start_date = "2021-01-01" #@param{type: "string"}
end_date = "2021-03-31" #@param{type: "string"}

In [ ]:
#@title define queries
base_sql = f"""
WITH
  base AS (
  SELECT
    pat.*,
    top_category,
    sub_category,
    naics_code
  FROM
    `{SG_table}` pat
  LEFT JOIN
    `{PLACES_table}`
  USING
    (placekey)
  WHERE date(date_range_start) between date("{start_date}") and date("{end_date}") ),

  join_sg_uc AS (
  SELECT
    sg.placekey,
    sg.location_name,
    case when sg.brands = "" or sg.brands is null then "NN" else sg.brands END brands,
    sg.region,
    sg.wkt_area_sq_meters,
    sg.parent_placekey,
    sg.date_range_start,
    sg.date_range_end,
    sg.top_category,
    sg.sub_category,
    naics_code || "_" || sub_category naics_code,
    sg.raw_visit_counts sg_raw_visit_counts,
    uc.raw_visit_counts uc_raw_visit_counts,
    sg.raw_visitor_counts sg_raw_visitor_counts,
    uc.raw_visitor_counts uc_raw_visitor_counts,
    sg.visitor_home_cbgs sg_visitor_home_cbgs,
    uc.visitor_home_cbgs uc_visitor_home_cbgs,
    sg.distance_from_home sg_distance_from_home,
    uc.distance_from_home uc_distance_from_home,
    sg.median_dwell sg_median_dwell,
    uc.median_dwell uc_median_dwell,
    sg.normalized_visits_by_state_scaling sg_normalized_visits_by_state_scaling,
    uc.normalized_visits_by_state_scaling uc_normalized_visits_by_state_scaling,
    sg.normalized_visits_by_total_visits sg_normalized_visits_by_total_visits,
    uc.normalized_visits_by_total_visits uc_normalized_visits_by_total_visits,
    sg.normalized_visits_by_total_visitors sg_normalized_visits_by_total_visitors,
    uc.normalized_visits_by_total_visitors uc_normalized_visits_by_total_visitors,
    sg.normalized_visits_by_region_naics_visits sg_normalized_visits_by_region_naics_visits,
    uc.normalized_visits_by_region_naics_visits uc_normalized_visits_by_region_naics_visits,
    --sg.normalized_visitors_by_region_naics_visits sg_normalized_visitors_by_region_naics_visits,
    --uc.normalized_visitors_by_region_naics_visits uc_normalized_visitors_by_region_naics_visits
  FROM
    base sg
  JOIN
    `{UC_table}` uc
  ON
    sg.placekey = uc.placekey
    AND DATE(sg.date_range_start) = DATE(uc.date_range_start)
  WHERE
    naics_code IN {industries}
    AND uc.date_range_start IS NOT NULL
    AND sg.date_range_start IS NOT NULL ), 

num_weeks_seen AS (
  SELECT placekey, COUNT(DISTINCT date_range_start) as w_c FROM 
  join_sg_uc
  GROUP BY placekey HAVING w_c > 7
)
SELECT 
  join_sg_uc.*
FROM join_sg_uc
INNER JOIN num_weeks_seen USING(placekey) 
"""



In [ ]:
#@title define functions

def add_trimmed_metrics(df):
  """
    We go through each column and for metrics we trim values low and above 1 quantile.
    This is applied on a per industry code basis and the resulting columns are "_trimmed"
  """

  for i in df.columns:
    
    if df[i].dtypes == "Int64" or df[i].dtypes == "float64":
      lb = df.groupby("naics_code")[i].quantile(0.01)
      ub = df.groupby("naics_code")[i].quantile(0.99)
      df = df.join(lb, on="naics_code", how="left", rsuffix="_lb")
      df = df.join(ub, on="naics_code", how="left", rsuffix="_ub")
      df[i + "_trimmed"]= df[i][(df[i] < df[i + "_ub"]) & (df[i] > df[i + "_lb"])]
      df.drop([i + "_lb", i + "_ub"], axis=1, inplace=True)

  return df

def plot_scatter(metric, df, time_inclusion = False, per_industry = "ALL"):
  """
    Scatterplot using plotly express for selected metric with hardcoded hover_over.
    If time_inclusion = True, then each weeks data will appear in the plot. So to
    speak a regression including time.
    If per_industry is defining "brand" or "naics_code" those will be used for color coding
    and ensure separate processing per brand or naics_code. if "ALL" the is no split and everything goes in one.
    Otherwise it will calculate the mean across time.
  """

  if per_industry != "ALL":
    if time_inclusion:
      plot_data = df.copy().groupby(["placekey", "date_range_start", "naics_code","brands", "wkt_area_sq_meters" ], as_index =False).mean()
    else:
      plot_data = df.copy().groupby(["placekey", "naics_code", "brands", "wkt_area_sq_meters"], as_index =False).mean()

    fig = px.scatter(
        plot_data, x='uc_' + metric, y='sg_' + metric, color=per_industry, opacity=0.8,
        trendline='ols', hover_data=["brands", "wkt_area_sq_meters"], template="unacast"
    )
  else:
    if time_inclusion:
      plot_data = df.copy().groupby(["placekey", "date_range_start","brands", "wkt_area_sq_meters" ], as_index =False).mean()
    else:
      plot_data = df.copy().groupby(["placekey", "brands", "wkt_area_sq_meters"], as_index =False).mean()

    fig = px.scatter(
        plot_data, x='uc_' + metric, y='sg_' + metric, opacity=0.8,
        trendline='ols', hover_data=["brands", "wkt_area_sq_meters"], template="unacast"
    )

  fig.show()


def get_all_metrics(df):
  """
    Get all unique metrics (int or float) from dataset
  """
  list_columns = []
  for i in df.columns:
    if df[i].dtypes == "Int64" or df[i].dtypes == "float64":
      if "sg_" in i or "uc_" in i:
        add_column = i[3:]
      else:
        add_column = i
      
      if add_column not in list_columns:
        list_columns.append(add_column) 
  
  list_columns.sort()
  return list_columns


def widget_scatter_plot(dataset):
    
    """
      Create widgets for metrics and analysis type to quickly change plots. 
      I stole Malachy's code and adjusted it :-) 
    """
    output = widgets.Output()
    dropdown_metric = widgets.Dropdown(options = get_all_metrics(dataset), 
                                        value = None, description='metrics:')
    dropdown_type = widgets.Dropdown(options = ["place", "time"], 
                                        value = None, description='analysis type:')
    dropdown_industry = widgets.Dropdown(options = ["ALL", "naics_code", "brands"], 
                                        value = None, description='per industry:')
    

    def output_scatter(metric, analysis_type, industry):
        try:
            time_inclusion = False
            if analysis_type == "time":
              time_inclusion = True

            output_data = plot_scatter(metric, dataset, time_inclusion, industry)
 
            with output:
                display(output_data)

        except Exception as e:
            IPython.display.clear_output(wait=True)
            print(e)
            display(input_widgets)
        
    def dropdown_metric_eventhandler(change):

        display(input_widgets)

        dropdown_metric = change.new
        output_scatter(metric = dropdown_metric, 
                       analysis_type = dropdown_type.value, 
                       industry = dropdown_industry.value)
        IPython.display.clear_output(wait=True)     

    def dropdown_type_eventhandler(change):

        display(input_widgets)

        dropdown_type = change.new
        output_scatter(metric = dropdown_metric.value, 
                       analysis_type = dropdown_type, 
                       industry = dropdown_industry.value)
        IPython.display.clear_output(wait=True)   

    def dropdown_industry_eventhandler(change):

        display(input_widgets)

        dropdown_industry = change.new
        output_scatter(metric = dropdown_metric.value, 
                       analysis_type = dropdown_type.value, 
                       industry = dropdown_industry)
        IPython.display.clear_output(wait=True) 

          
    dropdown_metric.observe(dropdown_metric_eventhandler, names='value')
    dropdown_type.observe(dropdown_type_eventhandler, names='value')
    dropdown_industry.observe(dropdown_industry_eventhandler, names='value')

    input_widgets = widgets.HBox([dropdown_industry, dropdown_type, dropdown_metric])
  
    display(input_widgets)
    IPython.display.clear_output(wait=True)    


def get_metric_summary(df, split_column="naics_code", top=None):
  """
   1. Get covariance matrix and fetch correlation coefficients per metric.
   2. create matrix with all correlations per placekey and split column
   3. Bring in the percentage of places having > .55 correlation.
   4. calculate percentage drop in volume from SG to UC
  """
  correlation_summary = pd.DataFrame()

  if top:
    top_split_column = list(df.loc[df[split_column] != "NN"].groupby(split_column).count().nlargest(top, "placekey").index)
    df = df.loc[df[split_column].isin(top_split_column)]

  if split_column:
    cov_matrix = df.groupby(split_column).corr()
  else:
    cov_matrix = df.corr()

  list_columns = get_all_metrics(df)

  list_columns.remove("wkt_area_sq_meters")
  list_columns.remove("wkt_area_sq_meters_trimmed")

  list_raw = [i for i in list_columns if "_trimmed" not in i]
  cov_matrix.reset_index(level=0, inplace=True)

  print("this is going to take a while since it runs correlations for all placekeys and naics codes...")

  temp_corr = df.groupby(["placekey", split_column]).corr().reset_index(level=1).reset_index(level=1)

  for metric in list_raw:
    temp_drop = (df.groupby(split_column)["sg_" + metric].mean() - df.groupby(split_column)["uc_" + metric].mean()) / df.groupby(split_column)["uc_" + metric].mean() * 100
    temp_drop_trimmed = (df.groupby(split_column)["sg_" + metric + "_trimmed"].mean() - df.groupby(split_column)["uc_" + metric + "_trimmed"].mean()) / df.groupby(split_column)["uc_" + metric + "_trimmed"].mean() * 100
    
    for n_code in cov_matrix[split_column].unique():
      correlation_summary = correlation_summary.append({'metrics': metric, 
                    split_column: n_code, 

                    # average correlation for raw and trimmed
                    'avg_correlation_raw': cov_matrix[cov_matrix[split_column] == n_code]["sg_" + metric]["uc_" + metric], 
                    'avg_correlation_trimmed': cov_matrix[cov_matrix[split_column] == n_code]["sg_" + metric + "_trimmed"]["uc_" + metric + "_trimmed"], 
                    
                    # get percentage of venues that are correlating > .55. the temp_corr comes with all possible combinations and needs some long filtering.
                    'percent_>.55_raw': round(np.sum(temp_corr[(temp_corr[split_column] == n_code) & (temp_corr['level_1'] == "sg_" + metric)]["uc_" + metric] > .55) / np.sum(temp_corr[(temp_corr[split_column] == n_code) & (temp_corr['level_1'] == "sg_" + metric)]["uc_" + metric] >= -1), 2) * 100, 
                    'percent_>.55_trimmed': round(np.sum(temp_corr[(temp_corr[split_column] == n_code) & (temp_corr['level_1'] == "sg_" + metric + "_trimmed")]["uc_" + metric + "_trimmed"] > .55) / np.sum(temp_corr[(temp_corr[split_column] == n_code) & (temp_corr['level_1'] == "sg_" + metric + "_trimmed")]["uc_" + metric + "_trimmed"] >= -1), 2) * 100, 
                    
                    # get mean percentage drop from SG to UC
                    'percent_volume_drop_raw': round(np.nanmean(temp_drop[n_code])), 
                    'percent_volume_drop_trimmed': round(np.nanmean(temp_drop_trimmed[n_code]))
                    }, ignore_index=True)
      
    print(f"{metric} is done!")
  return correlation_summary


In [ ]:
#@title run queries

query_job = client.query(base_sql, job_config=None)
df = query_job.to_dataframe()

#cast as float for some columns and add a trimmed version per industry
df['uc_distance_from_home'] = df['uc_distance_from_home'].astype("float64")
df['sg_distance_from_home'] = df['sg_distance_from_home'].astype("float64")
df = add_trimmed_metrics(df)



In [ ]:
#@title Widget
widget_scatter_plot(df)

In [ ]:
#@title get overall correlations of all metrics
grand_summary = get_metric_summary(df, "naics_code")
grand_summary.groupby(["metrics"]).mean()

In [ ]:
summary_by_naics_code = get_metric_summary(df, "naics_code")
summary_by_brand = get_metric_summary(df, "brands", top=50)
summary_by_region = get_metric_summary(df, "region")

In [ ]:
import plotly.express as px
import re
from cartoframes.viz import Map, Layer, color_continuous_style

In [ ]:
def metrics_per_group(summary, column, group, ndec=2):
    summary[column] = summary[column].apply(lambda x: np.round(x, ndec))
    op = summary[["metrics", group, column]].pivot(index="metrics", columns=group, values=column)
    op.columns = [re.sub("[0-9]+_", "", x) for x in op.columns]
    op = op.reindex(sorted(op.columns), axis=1)
    return op

In [ ]:
def plot_metrics_per_group(metrics_per_group, title):
    fig = px.imshow(metrics_per_group, 
                   text_auto=True, 
                   title = title,
                   template="unacast")
    fig.update_layout(
        yaxis={'title': ""},
        margin={"l": 80, "r": 80, "t": 100, "b": 80},
    )
    fig.show()


In [ ]:
plot_metrics_per_group(metrics_per_group(summary_by_naics_code, "avg_correlation_trimmed", "naics_code"), title="Average correlation trimmed by Naics Code for metrics")
plot_metrics_per_group(metrics_per_group(summary_by_naics_code, "percent_>.55_trimmed", "naics_code"), title= "Percent of placekeys with above .55 correlation by Naics Code for metrics")
plot_metrics_per_group(metrics_per_group(summary_by_naics_code, "percent_volume_drop_trimmed", "naics_code"), title="Percent volume drop trimmed by naics code for metrics")


In [ ]:
plot_metrics_per_group(metrics_per_group(summary_by_brand, "avg_correlation_trimmed", "brands"), title="Average correlation trimmed by brand for metrics")
plot_metrics_per_group(metrics_per_group(summary_by_brand, "percent_>.55_trimmed", "brands"), title= "Percent of placekeys with above .55 correlation by brand for metrics")
plot_metrics_per_group(metrics_per_group(summary_by_brand, "percent_volume_drop_trimmed", "brands"), title="Percent volume drop trimmed by brands for metrics")

In [ ]:
plot_metrics_per_group(metrics_per_group(summary_by_region, "avg_correlation_trimmed", "region"), title="Average correlation trimmed by region for metrics")
plot_metrics_per_group(metrics_per_group(summary_by_region, "percent_>.55_trimmed", "region"), title= "Percent of placekeys with above .55 correlation by region for metrics")
plot_metrics_per_group(metrics_per_group(summary_by_region, "percent_volume_drop_trimmed", "region"), title="Percent volume drop trimmed by region for metrics")

In [ ]:
get_region_polygons = client.query("""
SELECT iso.Abbreviation as region, geog FROM `uc-atlas.maps_us.states_iso` iso
LEFT JOIN `uc-atlas.maps_us.states` pol ON iso.State = pol.state_name
""").to_dataframe()

In [ ]:
def metric_by_region_map(summary, region_poly, column):
    maps = {}
    geog_df = summary.merge(region_poly, on="region", how="inner")
    for k, grp in geog_df.groupby("metrics"):
        maps[k] = Map([
                    Layer(
                        grp, 
                        geom_col="geog", 
                        style=color_continuous_style(value=column),
                        title = f"{column} for metric {k}"
                    )
                    ]
                )
    return maps

In [ ]:
metric_by_region = metric_by_region_map(summary_by_region, get_region_polygons, "avg_correlation_trimmed")

In [ ]:
metric_by_region["raw_visit_counts"]

In [ ]:
for n_code in grand_summary["naics_code"].unique():
  df_plot = grand_summary[grand_summary["naics_code"] == n_code].drop("naics_code", axis=1).set_index('metrics').transpose().stack().reset_index(level=0).copy()
  df_plot.reset_index(level=0, inplace=True)
  df_plot.columns = ["metrics", "features", "values"]

  fig = px.line_polar(df_plot, r="values", theta="features", color="metrics", line_close=True,
                    template="unacast")
  fig.show()

In [ ]:
#grand_summary.reset_index(level=0, inplace=True)
#grand_summary.columns = ["metrics", "avg_corr_raw", "avg_corr_trim", "%_>.55_raw", "%_>.55_trim", "%_vol_drop_raw", "%_vol_drop_trim"]
df_test = grand_summary.set_index('metrics').transpose().stack().reset_index(level=0).copy()
df_test.reset_index(level=0, inplace=True)
df_test.columns = ["metrics", "features", "values"]